## Imports

In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lcanalyzer import models

Params

In [17]:
plot_filter_labels = ["u", "g", "r", "i", "z", "y"]
plot_filter_colors = {
    "u": "#56b4e9",
    "g": "#008060",
    "r": "#ff4000",
    "i": "#850000",
    "z": "#6600cc",
    "y": "#000000",
}
plot_filter_symbols = {"u": "o", "g": "^", "r": "v", "i": "s", "z": "*", "y": "p"}
bands = 'ugrizy'
mag_col = 'psfMag'

## Read the data

In [13]:
lc_datasets = {}
lc_datasets["lsst"] = pd.read_pickle(os.path.join("data", "lsst_RRLyr.pkl"))
lc_datasets["kepler"] = pd.read_csv(os.path.join("data", "kepler_RRLyr.csv"))

In [19]:
lc_datasets["lsst"].head()

,band,ccdVisitId,coord_ra,coord_dec,objectId,psfFlux,psfFluxErr,psfMag,ccdVisitId2,band2,expMidptMJD,zeroPoint
0,y,1032263018,62.462569,-44.11336,1251384969897480052,-515.183603,1697.218490,NaN,1032263018,y,61100.069706,30.602301
1,y,1033987172,62.462569,-44.11336,1251384969897480052,3151.738459,1686.955775,22.653625,1033987172,y,61102.068464,30.606100
2,u,675163080,62.462569,-44.11336,1251384969897480052,183.449123,209.242045,25.741211,675163080,u,60582.247144,30.469101
3,y,443055067,62.462569,-44.11336,1251384969897480052,-704.848327,1624.400086,NaN,443055067,y,60215.203585,30.612801
4,u,466722002,62.462569,-44.11336,1251384969897480052,382.472233,278.926670,24.943500,466722002,u,60261.078221,30.461201


In [35]:
# Create an empty list where we will be storing our light curves
lcs = []
# For each observed object
for obj_id in lc_datasets["lsst"]["objectId"].unique():
    # Create an empty dict for the light curves of this object
    lc = {}
    lc['objectId'] = obj_id
    for b in bands:
        filt_band_obj = (lc_datasets["lsst"]["objectId"] == obj_id) & (
            lc_datasets["lsst"]["band"] == b
        )
        # The observations in each band are converted to lists and stored as dict elements
        lc[b+'_'+mag_col] = np.array(lc_datasets["lsst"][filt_band_obj][mag_col])
        # lc[b+'_'+time_col] = np.array(lc_datasets["lsst"][filt_band_obj][time_col])
    lcs.append(lc)
# Turn the list of dicts into a DataFrame    
lcs = pd.DataFrame.from_records(lcs)

In [36]:
for b in bands:
    lcs[b+'_'+mag_col+'_clean'] = lcs[b+'_'+mag_col].map(lambda l: np.where(np.isnan(l),0,l))

In [37]:
for b in bands:
    lcs[b+'_'+mag_col+'_clean_era'] = lcs.apply(
        lambda l: (
            l[b+'_'+mag_col][~np.isnan(l[b+'_'+mag_col])],
        ),
        axis=1,
        result_type="expand",
    )

In [38]:
lcs

,objectId,u_psfMag,g_psfMag,r_psfMag,i_psfMag,z_psfMag,y_psfMag,u_psfMag_clean,g_psfMag_clean,r_psfMag_clean,i_psfMag_clean,z_psfMag_clean,y_psfMag_clean,u_psfMag_clean_era,g_psfMag_clean_era,r_psfMag_clean_era,i_psfMag_clean_era,z_psfMag_clean_era,y_psfMag_clean_era
0,1251384969897480052,"[25.741210900246656, 24.94350022063739, nan, n...","[26.651280461495695, 25.95542807919974, 26.291...","[26.88813036987534, nan, 25.174556188807827, 2...","[24.622219025186332, 25.740813418843615, nan, ...","[24.76571729341986, 23.937013642017, 23.979465...","[nan, 22.653624571930173, nan, 23.861665693985...","[25.741210900246656, 24.94350022063739, 0.0, 0...","[26.651280461495695, 25.95542807919974, 26.291...","[26.88813036987534, 0.0, 25.174556188807827, 2...","[24.622219025186332, 25.740813418843615, 0.0, ...","[24.76571729341986, 23.937013642017, 23.979465...","[0.0, 22.653624571930173, 0.0, 23.861665693985...","[25.741210900246656, 24.94350022063739, 25.585...","[26.651280461495695, 25.95542807919974, 26.291...","[26.88813036987534, 25.174556188807827, 25.807...","[24.622219025186332, 25.740813418843615, 25.07...","[24.76571729341986, 23.937013642017, 23.979465...","[22.653624571930173, 23.861665693985756, 24.71..."
1,1251745609711384492,"[20.138613608325954, 20.08687354418428, 20.595...","[17.906176157090748, 18.401449236145567, 18.09...","[17.757833472878826, 17.979555432719586, 18.11...","[17.872576705276295, 17.936083857311147, 17.94...","[17.596257510993098, 17.90821223377898, 17.669...","[17.91065657820979, 17.58131790084475, 18.0128...","[20.138613608325954, 20.08687354418428, 20.595...","[17.906176157090748, 18.401449236145567, 18.09...","[17.757833472878826, 17.979555432719586, 18.11...","[17.872576705276295, 17.936083857311147, 17.94...","[17.596257510993098, 17.90821223377898, 17.669...","[17.91065657820979, 17.58131790084475, 18.0128...","[20.138613608325954, 20.08687354418428, 20.595...","[17.906176157090748, 18.401449236145567, 18.09...","[17.757833472878826, 17.979555432719586, 18.11...","[17.872576705276295, 17.936083857311147, 17.94...","[17.596257510993098, 17.90821223377898, 17.669...","[17.91065657820979, 17.58131790084475, 18.0128..."
2,1252299763571782414,"[18.801602946353793, 18.798748054955016, 18.80...","[18.008644608311656, 18.011763947918332, 17.99...","[17.769400071217, 17.77787920480292, 17.766536...","[17.70646770551845, 17.7027662708915, 17.70415...","[17.69208934046817, 17.702727082957217, 17.696...","[17.703955459952404, 17.701917865987845, 17.72...","[18.801602946353793, 18.798748054955016, 18.80...","[18.008644608311656, 18.011763947918332, 17.99...","[17.769400071217, 17.77787920480292, 17.766536...","[17.70646770551845, 17.7027662708915, 17.70415...","[17.69208934046817, 17.702727082957217, 17.696...","[17.703955459952404, 17.701917865987845, 17.72...","[18.801602946353793, 18.798748054955016, 18.80...","[18.008644608311656, 18.011763947918332, 17.99...","[17.769400071217, 17.77787920480292, 17.766536...","[17.70646770551845, 17.7027662708915, 17.70415...","[17.69208934046817, 17.702727082957217, 17.696...","[17.703955459952404, 17.701917865987845, 17.72..."
3,1251604872223041749,"[20.40735200568554, 20.393038732827815, 20.388...","[18.06648661371156, 18.061130305389447, 18.057...","[16.910867896515143, 16.9094019062832, 16.9022...","[16.39288178772588, 16.39094011806332, 16.3919...","[16.154658000299104, 16.15620783881299, 16.152...","[16.008204132883968, 16.014617067560383, 16.01...","[20.40735200568554, 20.393038732827815, 20.388...","[18.06648661371156, 18.061130305389447, 18.057...","[16.910867896515143, 16.9094019062832, 16.9022...","[16.39288178772588, 16.39094011806332, 16.3919...","[16.154658000299104, 16.15620783881299, 16.152...","[16.008204132883968, 16.014617067560383, 16.01...","[20.40735200568554, 20.393038732827815, 20.388...","[18.06648661371156, 18.061130305389447, 18.057...","[16.910867896515143, 16.9094019062832, 16.9022...","[16.39288178772588, 16.39094011806332, 16.3919...","[16